In [31]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/26 13:29:49 WARN Utils: Your hostname, tims-pc resolves to a loopback address: 127.0.1.1; using 192.168.178.46 instead (on interface wlp4s0)
22/02/26 13:29:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/26 13:29:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [9]:
df_pandas = pd.read_csv('head.csv')

In [10]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [11]:
df = spark.createDataFrame(df_pandas)

In [12]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [14]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [15]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')



In [16]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [17]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true)))

In [18]:
df = df.repartition(24)

In [19]:
df.write.parquet('fhvhv/2021/01/')

22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
22/02/26 13:45:42 WARN MemoryManager: Total allocation exceeds 95,

In [24]:
!ls fhvhv/2021/01

part-00000-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00001-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00002-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00003-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00004-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00005-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00006-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00007-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00008-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00009-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00010-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00011-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00012-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00013-d653fa80-fc88-4c9e-a36a-9051d9134278-c000.snappy.parquet
part-00014-d653fa80-fc88-4c9e-a36a-9051d9134278-

In [26]:
df = spark.read.parquet('fhvhv/2021/01/')

In [27]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [30]:
# lazy execution
df.select('pickup_datetime', 'PULocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+------------+
|    pickup_datetime|PULocationID|
+-------------------+------------+
|2021-01-01 21:18:35|          18|
|2021-01-01 23:25:49|          94|
|2021-01-02 19:51:39|         236|
|2021-01-02 00:02:37|         216|
|2021-01-01 16:34:07|         181|
|2021-01-02 20:45:23|         244|
|2021-01-01 01:51:25|          74|
|2021-01-02 00:47:54|         127|
|2021-01-01 04:26:22|          76|
|2021-01-02 22:26:02|          74|
|2021-01-01 14:37:09|         116|
|2021-01-01 14:12:24|          76|
|2021-01-02 12:43:11|          50|
|2021-01-02 11:18:54|          44|
|2021-01-01 01:51:55|          48|
|2021-01-01 16:00:14|         171|
|2021-01-01 03:16:48|          89|
|2021-01-02 11:59:23|          77|
|2021-01-01 11:05:38|         262|
|2021-01-02 15:24:47|          74|
+-------------------+------------+
only showing top 20 rows



In [34]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|          69|          41|
| 2021-01-01|  2021-01-01|         243|         116|
| 2021-01-01|  2021-01-01|          18|         259|
| 2021-01-01|  2021-01-01|          94|          32|
| 2021-01-02|  2021-01-02|         236|         143|
| 2021-01-02|  2021-01-02|         216|         216|
| 2021-01-01|  2021-01-01|         212|          32|
| 2021-01-01|  2021-01-01|         181|          75|
| 2021-01-02|  2021-01-02|          62|         225|
| 2021-01-02|  2021-01-02|         244|         243|
| 2021-01-01|  2021-01-01|          74|          39|
| 2021-01-02|  2021-01-02|         127|         136|
| 2021-01-01|  2021-01-01|          76|          89|
| 2021-01-02|  2021-01-02|          74|         116|
| 2021-01-02|  2021-01-02|          17|          61|
| 2021-01-01|  2021-01-01|         161|       

In [35]:
def custom_fct(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [38]:
udf = F.udf(custom_fct, returnType=types.StringType())